# IP8: Creation of Labelled Data
The RNN will be trained on spectrograms of the audio data from the created corpora. Since this process is computationally expensive and requires a lot of time. To speed up the iterations when training the RNN and get feedback faster, the input data (the spectrograms) are pre-computed and stored on disk. Also, the labels (the information about speech pauses) need to be encoded in a suitable format. This notebook describes how this is done.

Before we start, define a path to an empty directory with enough free storage where the labelled data can be stored:

In [ ]:
target_root = r'E:/'

As usual, let's do the imports and some helper functions before we start.

In [ ]:
%matplotlib inline
# %matplotlib notebook

import random
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, Audio
import ipywidgets as widgets
from IPython.display import HTML, Audio

from create_labelled_data import create_X_Y
from corpus_util import *
from audio_util import *
from webrtc_util import *

import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
from IPython.display import HTML, Audio
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

# import mpld3
# mpld3.enable_notebook()

from corpus_util import *
from audio_util import *

rl_corpus_root = os.path.join(target_root, 'readylingua-corpus')
ls_corpus_root = os.path.join(target_root, 'librispeech-corpus')

rl_data_root = os.path.join(target_root, 'readylingua-data')
ls_data_root = os.path.join(target_root, 'librispeech-data')

rl_corpus_path = os.path.join(rl_corpus_root, 'readylingua.corpus')
ls_corpus_path = os.path.join(ls_corpus_root, 'librispeech.corpus')

default_figsize = (50,20)
default_facecolor = 'white'
default_font = {'family': 'serif', 'weight': 'normal', 'size': 32}

plt.rc('font', **default_font)

def show_labelled_data(corpus_entry, data_root):
    display(HTML(f'<h3>{corpus_entry.name} (id={corpus_entry.id})</h3>'))
    display(HTML(f'{len(corpus_entry.speech_segments)} speech segments, {len(corpus_entry.pause_segments)} pause segments'))
    
    # audio data
    sample_rate, audio = corpus_entry.audio
    display(Audio(data=audio, rate=sample_rate))
    
    fig = plt.figure(figsize=default_figsize, facecolor=default_facecolor)
    
    # plot spectrogram
    freqs, times, spec = corpus_entry.spectrogram
    ax_spec, extent = show_spectrogram(freqs, times, spec, fig)
    
    # plot raw wave
    ax_wave = show_wave(audio, sample_rate, fig)
    
    # overlay pauses
    y = corpus_entry.labels
    if y is None:
        print(f'no labels for corpus entry with id={corpus_entry.id}')
    else:
        left, right, bottom, top = extent
        boundaries_frames = calculate_pause_boundaries_from_ground_truth(audio, y)
        show_pause_segments(ax_wave, boundaries_frames)
        show_pause_segments(ax_spec, (right-left) * boundaries_frames / len(audio))
        
    return ax_spec, ax_wave

def show_spectrogram(freqs, times, spec, fig=None):
    if not fig:
        fig = plt.figure(figsize=default_figsize, facecolor=default_facecolor)
    ax2 = fig.add_subplot(211)
    ax2.set_xlim(0, times.max())
    extent = [times.min(), times.max(), freqs.min(), freqs.max()]
    ax2.imshow(spec.T, aspect='auto', origin='lower', extent=extent)
    ax2.set_yticks(freqs[::16])
    ax2.set_xticks(times[::int(len(times)/10)])
    ax2.set_title('Spectrogram of ' + corpus_entry.audio_file)
    ax2.set_ylabel('Freqs in Hz')
    ax2.set_xlabel('Seconds')
    return ax2, extent

def show_wave(audio, sample_rate, fig=None, title=None):
    if not fig:
        fig = plt.figure(figsize=default_figsize, facecolor=default_facecolor)
    if not title:
        title = f'Raw wave of {corpus_entry.audio_file} with speech pauses'
    
    ax1 = fig.add_subplot(212)
    ax1.set_xlim(0, len(audio))
    ax1.set_title(title)
    ax1.set_ylabel('Amplitude')
    ax1.set_xlabel('Audio frames')
    ax1.plot(np.linspace(0, len(audio), len(audio)), audio)
    return ax1

def show_pause_segments(ax, boundaries, ymin=0, ymax=1, color='red'):
    for pause_start, pause_end in boundaries:
        ax.axvspan(pause_start, pause_end, ymin=ymin, ymax=ymax, color=color, alpha=0.8)
    
def calculate_pause_boundaries_from_ground_truth(x, y):
    """calculates the boundaries of pause segments in x given a label vector y.
    
    :y: numpy array of shape (1, T_y) containing the labels ("speech"/"no speech") for a RNN
    :x: numpy array of shape (T_x, ) containing the audio signal for a RNN
    """
    
    num_x = len(x)
    
    # pause boundaries as binary vector: [0,0,1,1,1,0,...]
    y = np.ravel(y)
    
    # pause boundaries as indices of 1-groups in y (start and end indices of group): [[2,4], ...]
    boundaries = np.flatnonzero(np.diff(np.r_[0,y,0]) != 0).reshape(-1,2) - [0,1]
    
    # pause boundaries as indices of 1-groups in x (calculated from relative position of frames in y)
    boundaries = len(x) * boundaries / len(y)
    
    # no fractional indices
    return boundaries.astype(int)

def calculate_boundaries_from_webrtc(segments, audio, sample_rate):
    boundaries = []
    for frames in segments:
        start_frame = frames[0].timestamp * sample_rate
        end_frame = (frames[-1].timestamp + frames[-1].duration) * sample_rate
        boundaries.append((start_frame, end_frame))
    return 2 * np.array(boundaries).astype(int)
    
def on_create_data_rl_button_click(sender):
    rl_target_root = os.path.join(target_root, 'readylingua-data')
    create_X_Y(ls_corpus, rl_target_root)
    
def on_create_data_ls_button_click(sender):
    ls_target_root = os.path.join(target_root, 'librispeech-data')
    create_X_Y(ls_corpus, ls_target_root)      
    
# UI elements
layout = widgets.Layout(width='250px', height='50px')
create_data_rl_btn = widgets.Button(description="Create labelled data for ReadyLingua", button_style='info', layout=layout, icon='download')
create_data_rl_btn.on_click(on_create_data_rl_button_click)
create_data_ls_btn = widgets.Button(description="Create labelled data for LibriSpeech", button_style='info', layout=layout, icon='download')
create_data_ls_btn.on_click(on_create_data_ls_button_click)

After having created the corpora from raw data we can now start creating labelled data (spectrograms and labels) for the RNN. This data is stored as numpy arrays whose dimensions partially depend on the proposed network architecture. 

The RNN is trained on audio data (sequence of frames) and will output whether a specific section in the audio signal is speech or pause (sequence of labels). Because both the input and the output is a sequence, it is a sequence-to-sequence model with a **many-to-many** architecture. This means we have the following values to consider:

* $T_x$: Number of sequence tokens in an individual sample. This value may be different for each sample!
* $T_y$: Number of sequence tokens in the output. This value is always the same for each sample but may be different from $T_x$

In the following sections the following variable names are used to denote the two components of the labelled data:

* `X`: The actual data, i.e. the spectrograms. One spectrogram is created per corpus entry and saved to disk. The saved data consists of three components:
  * `freqs`: The frequencies used in the spectrogram (array of shape $(161, 1)$)
  * `times`: The time steps used in the spectrogram (array of shape $(T_x, 1)$)
  * `spec`: The spectrogram data (array of shape $(T_x, 161)$)
  'freqs' and 'times' are only needed to plot the spectrogram along in a Cartesian coordinate system, where the time steps will be plotted along the x-axis and  frequencies along the y-axis. For training, only the `spec` part is needed.
* `Y`: The labels, i.e. the information about speech- or pause segments. The labels are encoded as 1-dimensional binary vectors of shape $(1, T_y)$. A speech segment will be encoded as a sequence of zeroes and a pause segment as a sequence of ones. Pause sections may contain some signal (e.g. background noise) but no spoken text from the transcript.

Let's load the created corpora to make them available to this notebook.

In [ ]:
rl_corpus = load_corpus(rl_corpus_path)
ls_corpus = load_corpus(ls_corpus_path)

## Train/Dev/Test split
The labelled data is split into subsets for training (_train-set_), parameter tuning (_dev-set_) and model evaluation (_test-set_). Since the corpora were create from different sources of raw data, they vary in size and probability distribution (number of languages, homogeneity of the recording quality, ratio of male vs. female speakers, presence of distortions like reverb or overdrive, and many more). Since the starting point for the creation of the corpus was so variable, different approaches were taken to split the corpus up into train-, dev- and test-set.

#### ReadyLingua corpus
The raw data exhibits a high variance with respect to relevant features (recording quality, length of samples, presence of distortion, ...). Since the corpus is rather small there may be only one sample for a specific feature value (e.g. only one recording with reverb). Therefore to keep things simple the split into train-, dev- and test-set was done with a 80/10/10-rule without closer examination of the underlying data. This might not result in an optimal split since it would be possible for example that all the female speakers will be put in one subset.

Improvements could be made by manually assigning each sample to a specific set by carefully inspecting the relevant features. The corpus could also be extended by creating synthetisized data, e.g. creating samples with reverb from the original samples. Because the LibriSpeech corpus looks much more promising at the moment, this time was not invested.

#### LibriSpeech corpus
The LibriSpeech raw data is already split into train-, dev- and test-set. Each chapter is read by a different speaker. Each speaker is only contained in one of the subsets. Efforts have been made to keep the different sets within the same probability distributions (regarding to accents, ratio of male/female speakers, ...). The information about the subset has been preserved when creating the corpora from raw data. To leverage the efforts made by the LibriSpeech project, the corresponding labelled data will be kept in the same subset.

---

You can explore the size of the subsets for each corpus by executing the cell below to see the number of samples (corpus entries) in each subset.

In [ ]:
ls_train, ls_dev, ls_test = ls_corpus.train_dev_test_split()
print(f'LibriSpeech corpus ({len(ls_corpus)} samples): #train-samples: {len(ls_train)}, #dev-samples: {len(ls_dev)}, #test-samples: {len(ls_test)}')

rl_train, rl_dev, rl_test = rl_corpus.train_dev_test_split()
print(f'ReadyLingua corpus ({len(rl_corpus)} samples): #train-samples: {len(rl_train)}, #dev-samples: {len(rl_dev)}, #test-samples: {len(rl_test)}')

##  From corpus entries to spectrograms
In order to train an RNN, each sample needs to be converted into some sort of sequence. In this case the samples are the audio files from the corpus entries and the sequences are their spectrograms. You can explore a random sample from the ReadyLingua corpus and visualize its spectrogram by executing the cell below. This will calculate a spectrogram for a corpus entry on-the-fly and plot it.

In [ ]:
corpus_entry = random.choice(ls_corpus)
rate, audio = corpus_entry.audio
freqs, times, spec = log_specgram(audio, rate)
show_spectrogram(freqs, times, spec)

Such a spectrogram is now created as a matrix `x` for every single corpus entry. A label vector `y` is also created for each corpus entry. This leaves us with two files for each entry. Since the spectrograms can become quite big, separate files are created for each entry. The files share a common naming pattern to identify their type (spectrogram or label), subset membership (train-, dev- or test-set) and corresponding corpus entry (ID of the corpus entry):

`{id}.{X|Y}.{train|dev|test}.npy`

For example the following files will be created for a corpus entry in the dev-set with ID `1234`:

```
1234.X.dev.npy
1234.Y.dev.npy
```

### Creating the spectrograms and labels

Creating the labelled data might take some time (around 15 minutes for the ReadyLingua corpus up to several hours for the LibriSpeech corpus). Click the button below to start computing the spectrograms and label vectors.

In [ ]:
display(widgets.HBox([create_data_rl_btn, create_data_ls_btn]))

## Exploring the labelled data
After the labelled data has been created, we can explore a sample by visualizing its spectrogram together with the information from the label vector. Execute the following cell to explore a random sample from the ReadyLingua corpus. In contrast to the spectrogram above this will not calculate the spectrogram on-the-fly but merely load the pre-computed files from disk. Together with the spectrogram the raw wave form will also be displayed. The labels (the information about pause segments) will be overlaid to both and will be visibile as r

In [ ]:
# corpus_entry = random.choice(rl_corpus)
# corpus_entry = rl_corpus[0]
corpus_entry = rl_corpus['news170524']
show_labelled_data(corpus_entry, rl_data_root)

Alternatively, we can detect speech pauses using a VAD (Voice Activity Detection) algorithm like the one from [WebRTC](https://webrtc.org/):

In [ ]:
corpus_entry = random.choice(rl_corpus)

sample_rate, audio = corpus_entry.audio
display(Audio(data=audio, rate=sample_rate))

# pause boundaries from raw data
y = corpus_entry.labels
original_boundaries = calculate_pause_boundaries_from_ground_truth(audio, y)

# pause boundaries from WebRTC
voiced_segments, unvoiced_segments = split_segments(corpus_entry)
webrtc_boundaries = calculate_boundaries_from_webrtc(unvoiced_segments, audio, sample_rate)

title = f'Raw wave of {corpus_entry.audio_file} with {len(original_boundaries)} original speech pauses (red) and {len(webrtc_boundaries)} speech pauses detected by WebRTC (green)'
ax_wave = show_wave(audio, sample_rate, title=title)
show_pause_segments(ax_wave, original_boundaries, ymax=0.5)
show_pause_segments(ax_wave, webrtc_boundaries, ymin=0.5, color='green')

We can now calculate how much the speech pauses detected by WebRTC coincide with the speech pauses from raw data:

In [ ]:
def calculate_overlaps(original_boundaries, webrtc_boundaries):
    boundaries = np.concatenate((original_boundaries, webrtc_boundaries))
    
    prev_start = -1
    prev_end = -1
    num_overlaps = 0
    overlaps = []
    for start, end in np.sort(boundaries, axis=0):
        if start <= prev_end: # we have an overlap
            overlap_start, overlap_end = (start, min(end, prev_end))
            
            # calculate number of frames that overlap
            overlap_frames = overlap_end - overlap_start
            
            # calculate degree of overlap
            segment_frames = prev_end - prev_start
            overlap_ratio = overlap_frames/segment_frames
            
            overlaps.append((overlap_frames, overlap_ratio))
        prev_start = start
        prev_end = end
        
    return overlaps

def compare_corpus_entry(corpus_entry, data_root):
    entry_stats = {}
    y = corpus_entry.labels
    original_boundaries = calculate_pause_boundaries_from_ground_truth(audio, y)
    
    entry_stats['raw'] = {}
    entry_stats['raw']['#pauses'] = len(corpus_entry.pause_segments)
    entry_stats['raw']['#pause_frames'] = sum([end_frame - start_frame for start_frame, end_frame in original_boundaries])
    
    voiced_segments, unvoiced_segments = split_segments(corpus_entry)
    webrtc_boundaries = calculate_boundaries_from_webrtc(unvoiced_segments, audio, sample_rate)    
    
    entry_stats['webrtc'] = {}
    entry_stats['webrtc']['#pauses'] = len(unvoiced_segments)
    entry_stats['webrtc']['#pause_frames'] = sum([end_frame - start_frame for start_frame, end_frame in webrtc_boundaries])

    overlaps = calculate_overlaps(original_boundaries, webrtc_boundaries)
    
    entry_stats['#overlaps'] = len(overlaps)
    entry_stats['#overlap_frames'] = np.sum([overlap_frames for overlap_frames, _ in overlaps])
    entry_stats['avg_overlap'] = np.mean([overlap_ratio for _, overlap_ratio in overlaps])
    
    return entry_stats
    
entry_stats = compare_corpus_entry(corpus_entry, rl_data_root)

num_pauses_raw = entry_stats['raw']['#pauses']
sum_pauses_raw = entry_stats['raw']['#pause_frames']
print(f'Length of all {num_pauses_raw} speech pauses in corpus entry (ground truth): {sum_pauses_raw}')

num_pauses_webrtc = entry_stats['webrtc']['#pauses']
sum_pauses_webrtc = entry_stats['webrtc']['#pause_frames']
print(f'Length of all {num_pauses_webrtc} speech pauses in corpus entry (WebRTC): {sum_pauses_webrtc}')

num_overlaps = entry_stats['#overlaps']
sum_overlaps = entry_stats['#overlap_frames']
print(f'WebRTC overlaps with original pauses {num_overlaps}/{num_pauses_raw} times ({100*num_overlaps/num_pauses_raw:.2f}%)')
print(f'WebRTC overlaps with original pauses in {sum_overlaps}/{sum_pauses_raw} frames ({100*sum_overlaps/sum_pauses_raw:.2f}%)')

avg_overlap = entry_stats['avg_overlap']
print(f'WebRTC pauses coincide with original pauses with {avg_overlap}')

We can further examine to what degree the speech pauses detected by WebRTC overlap with the speech pauses from the raw data for a whole corpus:

In [ ]:
from tqdm import tqdm

def compare_corpus(corpus, data_root):
    total_orig_pauses = 0
    total_orig_pause_frames = 0
    total_webrtc_pauses = 0
    total_webrtc_pause_frames = 0

    entry_stats = []
    for corpus_entry in tqdm(corpus, unit=' entries'):
        entry_stat = compare_corpus_entry(corpus_entry, data_root)
        entry_stats.append(entry_stat)

    total_overlaps = len(entry_stats)
    
    corpus_stats = {}
    
    corpus_stats['raw'] = {}
    corpus_stats['raw']['#pauses'] = sum(entry_stat['raw']['#pauses'] for entry_stat in entry_stats)
    corpus_stats['raw']['#pause_frames'] = sum(entry_stat['raw']['#pause_frames'] for entry_stat in entry_stats)
    
    corpus_stats['webrtc'] = {}
    corpus_stats['webrtc']['#pauses'] = sum(entry_stat['webrtc']['#pauses'] for entry_stat in entry_stats)
    corpus_stats['webrtc']['#pause_frames'] = sum(entry_stat['webrtc']['#pause_frames'] for entry_stat in entry_stats)
    
    corpus_stats['#overlaps'] = sum(entry_stat['#overlaps'] for entry_stat in entry_stats)
    corpus_stats['#overlap_frames'] = sum(entry_stat['#overlap_frames'] for entry_stat in entry_stats)
    corpus_stats['avg_overlap'] = np.mean([entry_stat['avg_overlap'] for entry_stat in entry_stats])
    
    num_pauses_raw = corpus_stats['raw']['#pauses']
    sum_pauses_raw = corpus_stats['raw']['#pause_frames']
    print(f'Length of all {num_pauses_raw} speech pauses in corpus (ground truth): {sum_pauses_raw}')

    num_pauses_webrtc = corpus_stats['webrtc']['#pauses']
    sum_pauses_webrtc = corpus_stats['webrtc']['#pause_frames']
    print(f'Length of all {num_pauses_webrtc} speech pauses in corpus (WebRTC): {sum_pauses_webrtc}')

    num_overlaps = corpus_stats['#overlaps']
    sum_overlaps = corpus_stats['#overlap_frames']

    print(f'WebRTC overlaps with original pauses {num_overlaps}/{num_pauses_raw} times ({100*num_overlaps/num_pauses_raw:.2f}%)')
    print(f'WebRTC overlaps with original pauses in {sum_overlaps}/{sum_pauses_raw} frames ({100*sum_overlaps/sum_pauses_raw:.2f}%)')

    return corpus_stats

corpus_stats = compare_corpus(rl_corpus, rl_data_root)
# corpus_stats = compare_corpus(ls_corpus, ls_data_root)